In [1]:
import sys
sys.path.insert(0, "../..")

In [2]:
import numpy as np
from matplotlib import pyplot as plt

from module.base.network import Network

from module.components.discrete_gaussian1D import DiscreteGaussian1D
from module.components.discrete_gaussian2D import DiscreteGaussian2D
from module.components.lawrence_dist import LawrenceDist

from module.simulation.meanfield import MeanField
from module.simulation.set_meanfield2 import SetMeanField2

import module.components.CONST as CONST

from module.simulation.meanfield2 import MeanField2

In [3]:
net = Network(3,3,1, [[0,0,0], [2,0,0], [0,2,0], [2,2,0]])
net.set_voltage_config([-2.625876307246630126e-01, -9.355253310870321679e-03, 1.687528930716121478e-02, -1.963316637927263186e-01], 5.020559674550297523e-03)
net.set_voltage_config([0.1,0,-0.02,0.01],0.03)

mf = MeanField(net)

g2 = DiscreteGaussian2D(phase_space_bounds_n=(-5,5), phase_space_bounds_m=(-5,5))
g1 = DiscreteGaussian1D(phase_space_min=-5, phase_space_max=5)

In [4]:
neighbour_table = net.get_nearest_neighbours(np.arange(0, net.N_particles))

covs = np.zeros((net.N_particles, 6))
def get_cov(i, j):
    table_index = np.where(neighbour_table[i] == j)[0]
    if table_index.shape[0] == 0:
        return 0

    return covs[i, table_index[0]]

def set_cov(i, j, value):
    table_index = np.where(neighbour_table[i] == j)[0]
    if table_index.shape[0] == 1:
        covs[i, table_index[0]] = value

    table_index = np.where(neighbour_table[j] == i)[0]
    if table_index.shape[0] == 1:
        covs[j, table_index[0]] = value

dcovs = np.zeros((net.N_particles, 6))
def get_dcov(i, j):
    table_index = np.where(neighbour_table[i] == j)[0]
    if table_index.shape[0] == 0:
        return 0

    return dcovs[i, table_index[0]]

def set_dcov(i, j, value):
    table_index = np.where(neighbour_table[i] == j)[0]
    if table_index.shape[0] == 1:
        dcovs[i, table_index[0]] = value

    table_index = np.where(neighbour_table[j] == i)[0]
    if table_index.shape[0] == 1:
        dcovs[j, table_index[0]] = value

---

In [5]:
def calc_effective_states(i, j):
    phase_space = g2.phase_space
    states = np.repeat(np.expand_dims(means, axis = [0, 1]), phase_space.shape[0], axis = 0)
    states = np.repeat(states, phase_space.shape[1], axis = 1)

    states[:,:,i] = phase_space[:,:,0]
    states[:,:,j] = phase_space[:,:,1]
    
    return states

---
$$
\langle I_{ij} \rangle
$$

In [6]:
def calc_R_island(i, j):
    states = calc_effective_states(i, j)
    rates = net.calc_rate_island(states, i, j)
    return rates

def calc_R_island_inv(i, j):
    states = calc_effective_states(i, j)
    rates = net.calc_rate_island(states, j, i)
    return rates

---
$$
\langle I_{ei} \rangle
$$

In [7]:
def calc_R_from_electrode(electrode_index):
    phase_space = g1.phase_space
    states = np.expand_dims(means, axis = 0)
    states = np.repeat(states, phase_space.shape[0], axis = 0)

    island_index = net.get_linear_indices(net.electrode_pos[electrode_index])
    states[:, island_index] = phase_space
    rates = net.calc_rate_from_electrode(states, electrode_index)
    return rates

def calc_R_to_electrode(electrode_index):
    phase_space = g1.phase_space
    states = np.expand_dims(means, axis = 0)
    states = np.repeat(states, phase_space.shape[0], axis = 0)

    island_index = net.get_linear_indices(net.electrode_pos[electrode_index])
    states[:, island_index] = phase_space
    rates = net.calc_rate_to_electrode(states, electrode_index)
    return rates

---
$$
\langle n_i I_{ei} \rangle
$$

In [8]:
def calc_nR_to_electrode(electrode_index):
    phase_space = g1.phase_space
    rates = calc_R_to_electrode(electrode_index)
    values = rates * phase_space 
    return values

def calc_nR_from_electrode(electrode_index):
    phase_space = g1.phase_space
    rates = calc_R_from_electrode(electrode_index)
    values = rates * phase_space 
    return values

---
$$
\langle n_i I_{ej} \rangle
$$

In [9]:
def calc_nR_from_electrode_2(i, electrode_index):
    phase_space = g2.phase_space
    island_index = net.get_linear_indices(net.electrode_pos[electrode_index])

    states = calc_effective_states(i, island_index)
    rates = net.calc_rate_from_electrode(states, electrode_index)

    return rates * phase_space[:,:,0]

def calc_nR_to_electrode_2(i, electrode_index):
    phase_space = g2.phase_space
    island_index = net.get_linear_indices(net.electrode_pos[electrode_index])

    states = calc_effective_states(i, island_index)
    rates = net.calc_rate_to_electrode(states, electrode_index)

    return rates * phase_space[:,:,0]

---
$$
\langle n_j I_{ij} \rangle
$$

In [10]:
def calc_nR_island(i, j):
    phase_space = g2.phase_space
    rates = calc_R_island(i, j)
    values = rates * phase_space[:,:,1] 
    return values

def calc_nR_island_inv(i, j):
    phase_space = g2.phase_space
    rates = calc_R_island_inv(i, j)
    values = rates * phase_space[:,:,1] 
    return values

---
$$
\langle n_i I_{ij} \rangle
$$

In [11]:
def calc_nR_island_alt(i, j):
    phase_space = g2.phase_space
    rates = calc_R_island(i, j)
    values = rates * phase_space[:,:,0] 
    return values

def calc_nR_island_inv_alt(i, j):
    phase_space = g2.phase_space
    rates = calc_R_island_inv(i, j)
    values = rates * phase_space[:,:,0] 
    return values

---
## Run

In [12]:
mf_means = mf.numeric_integration_solve(N = 30)

means = mf_means -0.5
vars = np.ones(net.N_particles)
covs = np.zeros((net.N_particles, 6))

In [27]:
dt = 0.07
for epoch in range(20):
    l_R = np.zeros(net.N_particles)
    r_R = np.zeros(net.N_particles)
    l_nR = np.zeros(net.N_particles)
    r_nR = np.zeros(net.N_particles)

    for i in range(net.N_particles):
        for j in neighbour_table[i]:
            if not j == -1: # all neighbour relations
                probs = g2.calc_prob(means[j], means[i], vars[j], vars[i],get_cov(i, j))
                l_R[i] += np.sum(probs * calc_R_island(j, i))
                l_nR[i] += np.sum(probs * calc_nR_island(j, i))
                r_R[i] += np.sum(probs * calc_R_island_inv(j, i))
                r_nR[i] += np.sum(probs * calc_nR_island_inv(j, i))
                
    l_R_electrodes = np.zeros(net.N_particles)
    r_R_electrodes = np.zeros(net.N_particles)
    l_nR_electrodes = np.zeros(net.N_particles)
    r_nR_electrodes = np.zeros(net.N_particles)

    for electrode_index, pos in enumerate(net.electrode_pos):
        i = net.get_linear_indices(pos)

        probs = g1.calc_prob(means[i], vars[i])
        l_R_electrodes[i] += np.sum(probs * calc_R_from_electrode(electrode_index)) 
        l_nR_electrodes[i] += np.sum(probs * calc_nR_from_electrode(electrode_index))

        r_R_electrodes[i] += np.sum(probs * calc_R_to_electrode(electrode_index)) 
        r_nR_electrodes[i] += np.sum(probs * calc_nR_to_electrode(electrode_index))

    # islands
    I_islands = l_R - r_R
    I_dag_islands = l_R + r_R

    nI_islands = l_nR - r_nR

    # electrodes
    I_electrodes = l_R_electrodes - r_R_electrodes
    I_dag_electrodes = l_R_electrodes + r_R_electrodes

    nI_electrodes = l_nR_electrodes - r_nR_electrodes

    # total
    I = I_islands + I_electrodes
    I_dag = I_dag_islands + I_dag_electrodes
    nI = nI_islands + nI_electrodes
    
    d_mean = I
    d_var = (2 * nI + I_dag) - 2 * means * I

    dcovs = np.zeros((net.N_particles, 6)) # reset dcovs
  
    for i in range(net.N_particles):
        for j in neighbour_table[i]:
            if not j == -1:
                probs = g2.calc_prob(means[i], means[j], vars[i], vars[j], get_cov(i, j))
                probs2 = g2.calc_prob(means[j], means[i], vars[j], vars[i], get_cov(i, j))
                island_indices = net.get_linear_indices(net.electrode_pos)


                # < ni Ij >
                dcov = means[i] * I_islands[j]
                dcov -= means[i] * np.sum(probs * (calc_R_island(i, j) - calc_R_island_inv(i, j)))
                dcov += np.sum(probs * (calc_nR_island_alt(i, j) - calc_nR_island_inv_alt(i, j)))

                electrode_index = np.where(island_indices == j)[0]
                if electrode_index.shape[0] == 1:
                    dcov += np.sum(probs * (calc_nR_from_electrode_2(i, electrode_index[0]) - calc_nR_to_electrode_2(i, electrode_index[0])))


                # < nj Ii >
                dcov += means[j] * I_islands[i]
                dcov -= means[j] * np.sum(probs2 * (calc_R_island(j, i) - calc_R_island_inv(j, i)))
                dcov += np.sum(probs2 * (calc_nR_island_alt(j, i) - calc_nR_island_inv_alt(j, i)))

                electrode_index = np.where(island_indices == i)[0]
                if electrode_index.shape[0] == 1:
                    dcov += np.sum(probs2 * (calc_nR_from_electrode_2(j, electrode_index[0]) - calc_nR_to_electrode_2(j, electrode_index[0])))

                # < I^dag_ij >
                dcov -= np.sum(probs * (calc_R_island(i, j) + calc_R_island_inv(i, j)))

                set_dcov(i, j, dcov - d_mean[i] * means[j] - means[i] * d_mean[j])

    means += dt * d_mean
    vars += dt * d_var
    covs += dt * dcovs

    vars = np.where(vars < 0, 0, vars)

    if  epoch % 1 == 0:
        print(np.abs(d_mean).max() , np.abs(d_var).max(), np.abs(dcovs).max())

0.08523289634338904 0.04925715381742525 0.11337158480147602
0.17732616896968073 0.2601115124887352 0.19539040011080716
0.1040763390766441 0.11793355871669664 0.1233823147457524
0.0847873306982006 0.062104312643195755 0.11804836766435425
0.06967080874975073 0.030061661340709304 0.11515626892238054
0.05766768073212908 0.01524103078641699 0.11367083931054695
0.04802284720196598 0.015948094129767824 0.11298994235821674
0.04018140709433454 0.016390731791092676 0.11276097635293292
0.033738943986900416 0.016684212865144224 0.11277895064317778
0.029829318012669193 0.016895771565183487 0.11292495325349748
0.026898484622961183 0.01706353816097356 0.11313052367728797
0.06595129096203078 0.10467316819324998 0.13998751745886656
0.010056872186053667 0.04370223534204953 0.1104681660201435
0.006877789012983193 0.03314593795161486 0.11076065595414727
0.03084043629666605 0.10753290952165778 0.1241425020796661
0.012373603249489551 0.06341238756255807 0.10763012108483952
0.015821897833660947 0.04461354870

In [28]:
def calc_expected_electrode_rates(electrode_index):
    r = calc_R_from_electrode(electrode_index) - calc_R_to_electrode(electrode_index)

    island_index = net.get_linear_indices(net.electrode_pos[electrode_index])
    probs = g1.calc_prob(means[island_index], vars[island_index])
    ex = np.sum(probs * r)

    return ex

In [33]:
means

array([ 1.04288827,  0.28198657, -0.41583115,  0.14286173, -0.08883153,
       -0.27882616, -0.7576052 , -0.40000635, -0.53825922])

In [34]:
mf_means

array([ 1.14150097,  0.30730644, -0.45897255,  0.17305524, -0.04159311,
       -0.18970848, -0.81723591, -0.25285378, -0.68534517])

In [35]:
vars

array([0.59983553, 0.41770078, 0.32000612, 0.48506631, 0.48641707,
       0.24886813, 0.45206578, 0.29028644, 0.25122507])

In [36]:
print(np.max(covs),np.min(covs)) 

0.27973461104147185 -0.3239193813594737


In [37]:
mf2 = MeanField2(net)

In [40]:
mf2.solve(N = 30, verbose = True)

convergence mean: 0.041564266483561874
convergence variances: 0.0015047791965758454
convergence covariances: 0.05475964431994855


In [41]:
mf2.means

array([ 1.04435951,  0.28543475, -0.41312669,  0.14373513, -0.07977314,
       -0.29354926, -0.75567032, -0.39935911, -0.50160199])

In [42]:
mf2.vars

array([0.59899772, 0.41893085, 0.32082247, 0.48467492, 0.49251492,
       0.24720308, 0.45329269, 0.30024278, 0.25086565])